In [1]:
import sys
sys.path.append('/home/shelunts/thesis/TabularSemantingParsing/')

In [4]:
from preprocess_inputs.parse_sql_one import get_schemas_from_json, Schema
from preprocess_inputs.process_sql import get_sql
from preprocess_inputs.get_tables import convert_fk_index, dump_db_json_schema
import json
import sqlite3
from os.path import isfile, isdir, join, split, exists, splitext
import pandas as pd
import nltk
import re
from src.eval.spider.evaluate import Evaluator
import pandas

In [7]:
ROOT_DIR = os.path.dirname(os.path.abspath(''))
DATA_DIR = ROOT_DIR + '/data/spider/database/isrecon/isrecon.sqlite'
SCHEMA_DIR = ROOT_DIR + '/data/spider/tables.json' 
SQL_DIR = ROOT_DIR + '/data/isrecon/isrecon/test_sql_pairs.sql'
DEV_DIR = ROOT_DIR + '/data/spider/dev.json'
PREDS_DIR1 = ROOT_DIR + '/model/isrecon_predictions/preds_isrecon_model1_final.txt'
PREDS_DIR2 = ROOT_DIR + '/model/isrecon_predictions/preds_isrecon_model2_final.txt'

Evaluation: ISRECON predictions

In [5]:
#instantiating the connection to the ISRECON DB
connection = sqlite3.connect(DATA_DIR)
crsr = connection.cursor()

In [6]:
crsr.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(crsr.fetchall())

[('dsr_count',), ('papers',), ('authors',), ('citations',), ('reference_citations',), ('papers_keywords',)]


In [8]:
with open(SQL_DIR, encoding='utf8') as f:
    l = [line.rstrip('\n') for line in f]
    queries = [v for i, v in enumerate(l) if i % 2 != 0]
predictions1 = open(PREDS_DIR1, 'r').read().split('\n')
predictions2 = open(PREDS_DIR2, 'r').read().split('\n')
# predictions1 = open('/home/shelunts/thesis/TabularSemantingParsing/model/model2_recon/predictions.txt', 'r').read().split('\n')
# predictions2 = open('/home/shelunts/thesis/TabularSemantingParsing/model/spider.bridge.lstm.meta.ts.ppl-0.85.2.dn.eo.feat.bert-large-uncased.xavier-1024-400-400-8-2-0.0005-inv-sqr-0.0005-4000-6e-05-inv-sqr-3e-05-4000-0.3-0.3-0.0-0.0-1-8-0.0-0.0-res-0.2-0.0-ff-0.4-0.0/predictions_isrecon_model2.txt', 'r').read().split('\n')

In [9]:
#executing results of ground truth queries and appending to the list
results_truths = [crsr.execute(i).fetchall() for i in queries]

In [10]:
#executing results of predictions of model 1 and appending to the list
results_predictions1 = []
for i in predictions1:
    try:
        prediction1 = crsr.execute(i).fetchall()
        results_predictions1.append(prediction1)
    except:
        results_predictions1.append("Error")
        pass

In [11]:
#executing results of predictions of model 2 and appending to the list
results_predictions2 = []
for i in predictions2:
    try:
        prediction2 = crsr.execute(i).fetchall()
        results_predictions2.append(prediction2)
    except:
        results_predictions2.append("Error")
        pass

In [12]:
#Model 1: checking if prediction is correct by comparing query output of ground truth queries vs predictions
accurate_predictions_indices1 = []
accurate_predictions_count1 = 0
prediction_status1 = []
idx = 0
for i in results_truths:
            if i == results_predictions1[idx]:
                accurate_predictions_indices1.append(idx)
                prediction_status1.append(True)
                accurate_predictions_count1 +=1
            elif results_predictions1[idx] == 'Error':
                prediction_status1.append('Error')
            else:
                prediction_status1.append(False)
            idx += 1

In [13]:
#Model 2: checking if prediction is correct by comparing query output of ground truth queries vs predictions
accurate_predictions_indices2 = []
accurate_predictions_count2 = 0
prediction_status2 = []
idx = 0
for i in results_truths:
            if i == results_predictions2[idx]:
                accurate_predictions_indices2.append(idx)
                prediction_status2.append(True)
                accurate_predictions_count2 +=1
            elif results_predictions2[idx] == 'Error':
                prediction_status2.append('Error')
            else:
                prediction_status2.append(False)
            idx += 1

In [14]:
print("Execution accuracy of the model 1: ", accurate_predictions_count1/len(predictions1))

Execution accuracy of the model 1:  0.46


In [15]:
print("Execution accuracy of the model 2: ", accurate_predictions_count2/len(predictions2))

Execution accuracy of the model 2:  0.63


In [16]:
#initializing Evaluator to estimate the difficulty of SQL queries
eval = Evaluator()

In [17]:
sqls_recon = json.load(open(DEV_DIR))
parsed_queries_isrecon = [i['sql'] for i in sqls_recon]
hardness_status_isrecon1 = [eval.eval_hardness(i) for i in parsed_queries_isrecon]

In [18]:
pd.Series(hardness_status_isrecon1).value_counts(normalize=True)

medium    0.51
easy      0.19
hard      0.15
extra     0.15
dtype: float64

In [19]:
evaluation_df = pd.DataFrame(data = {'SQL true': queries, 'SQL pred 1': predictions1, 'SQL pred 2': predictions2, 'Hardness level': hardness_status_isrecon1, 'Prediction status 1': prediction_status1, 'Prediction status 2': prediction_status2})
evaluation_df.head()

,SQL true,SQL pred 1,SQL pred 2,Hardness level,Prediction status 1,Prediction status 2
0,SELECT distinct papers.title from papers join ...,SELECT papers.title FROM authors JOIN papers O...,SELECT DISTINCT papers.title FROM papers JOIN ...,medium,False,True
1,SELECT COUNT(distinct papers.title) from paper...,SELECT COUNT(*) FROM authors JOIN papers ON pa...,SELECT COUNT(*) FROM papers JOIN authors ON au...,medium,False,False
2,"select authors.firstname, authors.surname from...","SELECT DISTINCT COUNT(*), papers.authors FROM ...",SELECT authors.firstname FROM papers JOIN auth...,extra,False,False
3,SELECT count(distinct papers.article_id) from ...,SELECT COUNT(*) FROM authors JOIN papers ON pa...,SELECT COUNT(DISTINCT papers.title) FROM paper...,medium,True,True
4,SELECT papers.title from papers join authors o...,SELECT papers.title FROM authors JOIN papers O...,SELECT papers.title FROM papers JOIN authors O...,medium,True,True


In [20]:
#distribution of accurate predictions according to the hardness level of SQL
pd.crosstab(index=evaluation_df['Prediction status 1'], columns=evaluation_df['Hardness level'],normalize='columns').round(3)*100

Hardness level,easy,extra,hard,medium
Prediction status 1,,,,
False,21.1,93.3,46.7,52.9
True,73.7,6.7,53.3,45.1
Error,5.3,0.0,0.0,2.0


In [57]:
#distribution of accurate predictions according to the hardness level of SQL
pd.crosstab(index=evaluation_df['Prediction status 2'], columns=evaluation_df['Hardness level'],normalize='columns').round(3)*100

Hardness level,easy,extra,hard,medium
Prediction status 2,,,,
False,26.3,26.7,14.3,50.0
True,73.7,73.3,85.7,50.0
